In [ ]:
# Import several packages that will be used throughout

# numeric packages
import numpy as np
import scipy
import scipy.io
import pandas as pd
import h5py

# filesystem and OS
import sys, os, time
import glob

# plotting
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

# these magics ensure that external modules that are modified are also automatically reloaded
%load_ext autoreload
%autoreload 2

In [ ]:
import keras
import tensorflow as tf

In [ ]:
sys.path.append("/home/adalbert/nbserver/urban-environments/keras-utils/")
sys.path.append("/home/adalbert/nbserver/urban-environments/keras-models/")

from multi_gpu import make_parallel
import keras_utils as ku
from vgg16 import vgg16

In [ ]:
workdir = "/home/adalbert/nbserver/tf-workspace/deepsat-experiments/"
os.chdir(workdir)

# Import data and set up batching 

### Set up batching

In [ ]:
BATCH_SIZE = 100

train_dir = "/home/adalbert/data/DeepSat/img/train/"
test_dir = "/home/adalbert/data/DeepSat/img/test/"

In [ ]:
# Generator for preprocessing images for data augmentation

from keras.preprocessing.image import ImageDataGenerator

def apply_mean(image_data_generator):
    """Subtracts the VGG dataset mean"""
    image_data_generator.mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3, 1, 1))

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=[1,1.2],
        vertical_flip=True,
        rotation_range=15,
        horizontal_flip=True)
apply_mean(train_datagen)


# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)
apply_mean(test_datagen)

train_generator = train_datagen.flow_from_directory(train_dir, \
                                         batch_size=BATCH_SIZE,
                                         target_size=(224,224),
                                         shuffle=True)
    
# this is a similar generator, for validation data
test_generator = test_datagen.flow_from_directory(test_dir, \
                                         batch_size=BATCH_SIZE,
                                         target_size=(224,224),
                                         shuffle=True)

# get class labels
class2ind = train_generator.class_indices
ind2class = {v:k for k,v in class2ind.iteritems()}

N_CLASSES = len(ind2class)

In [ ]:
for Xbatch, ybatch in train_generator:
    print Xbatch.shape, ybatch.shape, Xbatch.min(), Xbatch.max()
    break

# Define architectures to use
Load weights from model pretrained on ImageNet

#### VGG-16, pre-trained on ImageNet
Note that the weights are trained on BGR data for this architecture (probably following Caffe's convention). It turns out that they're a good enough starting point even if using RGB images.

In [ ]:
# build the convolutional base of the VGG16 network
model = vgg16(n_classes=N_CLASSES, input_shape=(224,224,3), fcn=False)

weights_file = "../vgg16_weights.h5"
model = ku.load_weights_into_model(model, weights_file, transpose_conv=True, 
                        layers_to_skip=["dense8"])

freeze_layers = [] #['conv1', 'conv2']
for l in model.layers:
    l.trainable = len([x for x in freeze_layers if x in l.name])==0 and \
                    len(l.get_weights())>0

for l in model.layers:
    print l.name, [x.sum() for x in l.get_weights()], l.trainable

#### ResNet initialized with ImageNet weights

In [ ]:
sys.path.append("/home/adalbert/nbserver/DeepGold/deep-learning-models/")

from keras.layers import Flatten, Dense
from keras.models import Model
from resnet50 import ResNet50
from keras.preprocessing import image
from imagenet_utils import preprocess_input, decode_predictions

model = ResNet50(weights='imagenet', include_top=True)

model.layers.pop() # Get rid of the classification layer
model.outputs = [model.layers[-1].output]
model.output_layers = [model.layers[-1]] # added this line in addition to zo7 solution
model.layers[-1].outbound_nodes = []

newClassificationLayer = Dense(N_CLASSES, activation='softmax')(model.layers[-1].output)
model = Model(input=model.input, output=newClassificationLayer)


#### Load previously-saved model

In [ ]:
# # load model from checkpoint
# model_file = "vgg16-deepsat-best-checkpoint.h5"

# # with tf.device('/cpu:0'):
# # load model and weights
# model = keras.models.load_model(model_file)

Spread model computation on multiple GPUs

In [ ]:
GPUS = [0,1,2,3]

# this uses the TensorFlow backend to spread computation on multiple GPUs
model_gpu = make_parallel(model, GPUS)

In [ ]:
tf.__version__, keras.__version__

# Fine-tune model

Define model behavior with callbacks and compile

In [ ]:
from keras.callbacks import ModelCheckpoint

# callback for a custom learning rate decay schedule

LR_DECAY_PER_EPOCH = 2.0 #1.1
BASE_LR = 1

lr_scheduler = lambda epoch: BASE_LR * LR_DECAY_PER_EPOCH**(-(epoch/10))
lr_decay_callback = keras.callbacks.LearningRateScheduler(lr_scheduler)

# callback to checkpoint best model
model_checkpoint_callback = ModelCheckpoint("resnet-deepsat-imagenet-best-checkpoint.h5", monitor='val_acc', \
                                      verbose=1, save_best_only=True, mode='max')

# compile model
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
model_gpu.compile(loss='categorical_crossentropy', \
              metrics=['accuracy'],\
              optimizer=Adadelta(lr=BASE_LR))

Compile and train model

In [ ]:
# Logs to TensorBoard, new one for each run

log_path_tensorboard = "./logs/"

from keras.callbacks import TensorBoard
import time

now = time.strftime("%c")
tensorboard_callback_fn = TensorBoard(log_dir=log_path_tensorboard + now, \
                                histogram_freq=1, \
                                write_graph=True, \
                                write_images=False)

# Train model

history = model_gpu.fit_generator(
            train_generator,
            samples_per_epoch=2000,
            nb_epoch=50,
            validation_data=test_generator,
            callbacks = [tensorboard_callback_fn, lr_decay_callback, \
                         model_checkpoint_callback],
            nb_val_samples=1000)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,4))
ax[0].plot(history.history['loss'], label="train")
ax[0].plot(history.history['val_loss'], label="test")
ax[0].set_title("Loss", fontsize=14)
ax[0].set_xlabel("Epoch")
ax[0].legend(loc="best")
ax[1].plot(history.history['acc'], label="train")
ax[1].plot(history.history['val_acc'], label="test")
ax[1].set_title("Accuracy", fontsize=14)
ax[1].set_xlabel("Epoch")
ax[1].legend(loc="best")
plt.show()

In [ ]:
tf.__version__